In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

import os
import pandas as pd
os.getcwd()


from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import TargetEncoder, LabelEncoder

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import pandas as pd
import time
import pickle
from xgboost import XGBClassifier

import pandas as pd
import time
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split

NUMERIC_COLS = [
    "TP_FAIXA_ETARIA",
    "TP_ESTADO_CIVIL",
    "TP_COR_RACA",
    "TP_NACIONALIDADE",
    "TP_ST_CONCLUSAO",
    "TP_ANO_CONCLUIU",
    "IN_TREINEIRO",
    "CO_MUNICIPIO_PROVA",
    "CO_UF_PROVA",
    "TP_PRESENCA_CN",
    "TP_PRESENCA_CH",
    "TP_PRESENCA_LC",
    "TP_PRESENCA_MT",
    "NU_NOTA_CN",
    "NU_NOTA_CH",
    "NU_NOTA_LC",
    "NU_NOTA_MT",
    "TP_LINGUA",
    "NU_NOTA_REDACAO",
    "NU_NOTA_COMP1",
    "NU_NOTA_COMP2",
    "NU_NOTA_COMP3",
    "NU_NOTA_COMP4",
    "NU_NOTA_COMP5",
    "TP_ESCOLA"
]

CATEGORICAL_COLS = [
    "TP_SEXO",
    "Q001",
    "Q002",
    "Q003",
    "Q004",
    "Q007",
    "Q008",
    "Q009",
    "Q010",
    "Q011",
    "Q012",
    "Q013",
    "Q014",
    "Q015",
    "Q016",
    "Q017",
    "Q018",
    "Q019",
    "Q020",
    "Q021",
    "Q022",
    "Q023",
    "Q024",
    "Q025",
    "NO_MUNICIPIO_PROVA",
    "TP_STATUS_REDACAO",
    "SG_UF_PROVA"
]


def generate_pipeline():
    numeric_transformer = Pipeline(steps=[("scaler", MinMaxScaler())])

    categorical_transformer = Pipeline(steps=[("encoder", OrdinalEncoder())])

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, NUMERIC_COLS),
            ("cat", categorical_transformer, CATEGORICAL_COLS),
            ("num2", numeric_transformer, CATEGORICAL_COLS)
        ]
    )

    return Pipeline(steps=[("preprocessor", preprocessor)])

2024-06-16 15:17:20.033726: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-16 15:17:21.021389: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-16 15:17:23.467290: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-16 15:17:27.203319: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df_microdados = pd.read_parquet('../../tcc-results/data-results/data_prepared_enem_2022.parquet')


map_renda = {
    "A": ["P", "Q"], # 15 A 25 SALÁRIOS
    "B": ["N", "O"], # 10 A 15 SALÁRIOS
    "C": ["J", "K", "L", "M"], # 5 A 10 SALÁRIOS
    "D": ["E", "F", "G", "H", "I"], # 2 A 5 SALÁRIOS
    "E":  ["A", "B", "C"], # ATÉ 2 SALÁRIOS
}

# Função para fazer o mapeamento inverso
def map_values(value, mapping):
    for key, values in mapping.items():
        if value in values:
            return key
    return value

# Aplicar a função de mapeamento na coluna do DataFrame
df_microdados['faixa_renda_familiar'] = df_microdados['faixa_renda_familiar'].apply(map_values, mapping=map_renda)

def encode_target(y):
    encoder = LabelEncoder()
    y_encoded = encoder.fit_transform(y)
    return y_encoded, encoder

X = df_microdados.drop('faixa_renda_familiar', axis=1)
y = df_microdados['faixa_renda_familiar']
y_encoded, encoder = encode_target(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [3]:
from keras.layers import Input, Dense, Dropout, BatchNormalization, LeakyReLU
from keras.models import Model

# Dimensionalidade dos dados de entrada
input_dim = X_train_method.shape[1]
# Dimensionalidade do espaço latente (reduzida)
encoding_dim = 2  # Reduzir para 2 dimensões para visualização

# Definir o autoencoder com mais camadas e melhorias
input_data = Input(shape=(input_dim,))
encoded = Dense(128)(input_data)
encoded = BatchNormalization()(encoded)
encoded = LeakyReLU(alpha=0.1)(encoded)
encoded = Dropout(0.2)(encoded)
encoded = Dense(64)(encoded)
encoded = BatchNormalization()(encoded)
encoded = LeakyReLU(alpha=0.1)(encoded)
encoded = Dropout(0.2)(encoded)
encoded = Dense(32)(encoded)
encoded = BatchNormalization()(encoded)
encoded = LeakyReLU(alpha=0.1)(encoded)
encoded = Dropout(0.2)(encoded)
encoded = Dense(encoding_dim, activation='linear')(encoded)

decoded = Dense(32)(encoded)
decoded = BatchNormalization()(decoded)
decoded = LeakyReLU(alpha=0.1)(decoded)
decoded = Dropout(0.2)(decoded)
decoded = Dense(64)(decoded)
decoded = BatchNormalization()(decoded)
decoded = LeakyReLU(alpha=0.1)(decoded)
decoded = Dropout(0.2)(decoded)
decoded = Dense(128)(decoded)
decoded = BatchNormalization()(decoded)
decoded = LeakyReLU(alpha=0.1)(decoded)
decoded = Dropout(0.2)(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# Modelo autoencoder
autoencoder = Model(input_data, decoded)

# Modelo encoder
encoder = Model(input_data, encoded)

# Compilar o modelo
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Treinar o modelo
autoencoder.fit(X_train_method, X_train_method,
                epochs=50,  # Aumentar o número de épocas para melhor convergência
                batch_size=256,  # Usar um batch size maior para eficiência
                shuffle=True,
                validation_data=(X_test_transformed, X_test_transformed))

# Usar o encoder para reduzir a dimensionalidade dos dados de teste
encoded_data = encoder.predict(X_test_transformed)

# Mostrar as primeiras linhas dos dados reduzidos
print(encoded_data[:5])


ValueError: could not convert string to float: 'M'

In [ ]:
encoded_train = pd.DataFrame(encoder.predict(train_scaled))
encoded_train = encoded_train.add_prefix('feature_')

encoded_test = pd.DataFrame(encoder.predict(test_scaled))
encoded_test = encoded_test.add_prefix('feature_')